In [ ]:
#|default_exp funccall

# funccall source

In [ ]:
#| exports
import inspect, json, ast
from collections import abc
from fastcore.utils import *
from fastcore.docments import docments
from typing import get_origin, get_args, Dict, List, Optional, Tuple, Union, Any
from types import UnionType
from typing import get_type_hints
from inspect import Parameter, Signature
from decimal import Decimal
from uuid import UUID

In [ ]:
#|hide
from fastcore.test import *

In [ ]:
#| export
empty = inspect.Parameter.empty

## Function calling

### Function to schema

Many LLMs do function calling (aka tool use) by taking advantage of JSON schema.

We'll use [docments](https://fastcore.fast.ai/docments.html) to make getting JSON schema from Python functions as ergonomic as possible. Each parameter (and the return value) should have a type, and a docments comment with the description of what it is. Here's an example:

In [ ]:
def silly_sum(
    a:int, # First thing to sum
    b:int=1, # Second thing to sum
    c:list[int]=None, # A pointless argument
) -> int: # The sum of the inputs
    "Adds a + b."
    return a + b

This is what `docments` makes of that:

In [ ]:
d = docments(silly_sum, full=True)
d

```python
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'First thing to sum'},
  'b': {'anno': <class 'int'>, 'default': 1, 'docment': 'Second thing to sum'},
  'c': {'anno': list[int], 'default': None, 'docment': 'A pointless argument'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'The sum of the inputs'}}
```

Note that this is an [AttrDict](https://fastcore.fast.ai/basics.html#attrdict) so we can treat it like an object, *or* a dict:

In [ ]:
d.a.docment, d['a']['anno']

('First thing to sum', int)

In [ ]:
#| exports
def _types(t:type)->tuple[str,Optional[str]]:
    "Tuple of json schema type name and (if appropriate) array item name."
    if t is empty: raise TypeError('Missing type')
    tmap = {int:"integer", float:"number", str:"string", bool:"boolean", list:"array", dict:"object"}
    tmap.update({k.__name__: v for k, v in tmap.items()})
    if getattr(t, '__origin__', None) in (list,tuple):
        args = getattr(t, '__args__', None)
        item_type = "object" if not args else tmap.get(t.__args__[0].__name__, "object")
        return "array", item_type
    # if t is a string like 'int', directly use the string as the key
    elif isinstance(t, str): return tmap.get(t, "object"), None
    # if t is the type itself and a container
    elif get_origin(t): return tmap.get(get_origin(t).__name__, "object"), None
    # if t is the type itself like int, use the __name__ representation as the key
    else: return tmap.get(t.__name__, "object"), None

This internal function is needed to convert Python types into JSON schema types.

In [ ]:
_types(list[int]), _types(int), _types('int')

(('array', 'integer'), ('integer', None), ('integer', None))

In [ ]:
_types(List[int]), _types(Optional[str]), _types(str | None), _types(Tuple[str, int])

(('array', 'integer'), ('object', None), ('object', None), ('array', 'string'))

Note the current behavior:

- ignores all but the first argument for tuples
- union types map to object which is a stand-in for arbitrary types

These and other approximations may require further refinement in the future.

Will also convert custom types to the `object` type.

In [ ]:
class Custom: a: int
_types(list[Custom]), _types(Custom)

(('array', 'object'), ('object', None))

In [ ]:
#| exports
def _param(
    name, # param name
    info, # dict from docments
    evalable=False): # stringify defaults that can't be literal_eval'd?
    "json schema parameter given `name` and `info` from docments full dict"
    paramt,itemt = _types(info.anno)
    pschema = dict(type=paramt, description=info.docment or "")
    if itemt: pschema["items"] = {"type": itemt}
    if info.default is not empty:
        if evalable:
            try: ast.literal_eval(repr(info.default))
            except: pschema["default"] = str(info.default)
            else: pschema["default"] = info.default
        else: pschema["default"] = info.default
    return pschema

This private function converts a key/value pair from the `docments` structure into the `dict` that will be needed for the schema.

In [ ]:
n,o = first(d.items())
print(n,'//', o)
_param(n, o)

a // {'docment': 'First thing to sum', 'anno': <class 'int'>, 'default': <class 'inspect._empty'>}


{'type': 'integer', 'description': 'First thing to sum'}

In [ ]:
n,o

('a',
 {'docment': 'First thing to sum', 'anno': int, 'default': inspect._empty})

In [ ]:
d

```python
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'First thing to sum'},
  'b': {'anno': <class 'int'>, 'default': 1, 'docment': 'Second thing to sum'},
  'c': {'anno': list[int], 'default': None, 'docment': 'A pointless argument'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'The sum of the inputs'}}
```

In [ ]:
#| export
custom_types = {Path, bytes, Decimal, UUID}

def _handle_type(t, defs):
    "Handle a single type, creating nested schemas if necessary"
    if t is NoneType: return {'type': 'null'}
    if t in custom_types: return {'type':'string', 'format':t.__name__}
    if t in (dict, list, tuple, set): return {'type': _types(t)[0]}
    if isinstance(t, type) and not issubclass(t, (int, float, str, bool)) or inspect.isfunction(t):
        defs[t.__name__] = _get_nested_schema(t)
        return {'$ref': f'#/$defs/{t.__name__}'}
    return {'type': _types(t)[0]}

In [ ]:
_handle_type(int, None), _handle_type(Path, None)

({'type': 'integer'}, {'type': 'string', 'format': 'Path'})

In [ ]:
#| export
def _is_container(t):
    "Check if type is a container (list, dict, tuple, set, Union)"
    origin = get_origin(t)
    return origin in (list, dict, tuple, set, Union, UnionType) if origin else False

def _is_parameterized(t):
    "Check if type has arguments (e.g. list[int] vs list, dict[str, int] vs dict)"
    return _is_container(t) and (get_args(t) != ())

In [ ]:
assert _is_parameterized(list[int]) == True
assert _is_parameterized(int) == False
assert _is_container(list[int]) == True
assert _is_container(dict[str, int]) == True
assert _is_container(int) == False

For union and optional types, `Union` covers older `Union[str]` syntax while `UnionType` covers 3.10+ `str | None` syntax.

In [ ]:
def _example_new_unioin(opt_tup: str | None):
    pass

d = docments(_example_new_unioin, full=True)
anno1 = first(d.items())[1].anno
(anno1, get_origin(anno1), get_args(anno1))

(str | None, types.UnionType, (str, NoneType))

In [ ]:
def _example_old_union(opt_tup: Union[str, type(None)] =None):
    pass

d = docments(_example_old_union, full=True)
anno2 = first(d.items())[1].anno
(anno2, get_origin(anno2), get_args(anno2))

(typing.Optional[str], typing.Union, (str, NoneType))

Support for both union types is part of the broader container handling:

In [ ]:
#| export
def _handle_container(origin, args, defs):
    "Handle container types like dict, list, tuple, set, and Union"
    if origin is Union or origin is UnionType:
        return {"anyOf": [_handle_type(arg, defs) for arg in args]}
    if origin is dict:
        value_type = args[1].__args__[0] if hasattr(args[1], '__args__') else args[1]
        return {
            'type': 'object',
            'additionalProperties': (
                {'type': 'array', 'items': _handle_type(value_type, defs)}
                if hasattr(args[1], '__origin__') else _handle_type(args[1], defs)
            )
        }
    elif origin in (list, tuple, set):
        schema = {'type': 'array', 'items': _handle_type(args[0], defs)}
        if origin is set:
            schema['uniqueItems'] = True
        return schema
    return None

In [ ]:
#| export
def _process_property(name, obj, props, req, defs, evalable=False):
    "Process a single property of the schema"
    p = _param(name, obj, evalable=evalable)
    props[name] = p
    if obj.default is empty: req[name] = True

    if _is_container(obj.anno) and _is_parameterized(obj.anno):
        p.update(_handle_container(get_origin(obj.anno), get_args(obj.anno), defs))        
    else:
        # Non-container type or container without arguments
        p.update(_handle_type(obj.anno, defs))

In [ ]:
#| export
def _get_nested_schema(obj, evalable=False, skip_hidden=False):
    "Generate nested JSON schema for a class or function"
    d = docments(obj, full=True)
    props, req, defs = {}, {}, {}

    for n, o in d.items():
        if n != 'return' and n != 'self' and not (skip_hidden and n.startswith('_')):
            _process_property(n, o, props, req, defs, evalable=evalable)

    tkw = {}
    if isinstance(obj, type): tkw['title']=obj.__name__
    schema = dict(type='object', properties=props, **tkw)
    if req: schema['required'] = list(req)
    if defs: schema['$defs'] = defs
    return schema

In [ ]:
# Test primitive types
defs = {}
assert _handle_type(int, defs) == {'type': 'integer'}
assert _handle_type(str, defs) == {'type': 'string'}
assert _handle_type(bool, defs) == {'type': 'boolean'}
assert _handle_type(float, defs) == {'type': 'number'}

# Test custom class
class TestClass:
    def __init__(self, x: int, y: int): store_attr()

result = _handle_type(TestClass, defs)
assert result == {'$ref': '#/$defs/TestClass'}
assert 'TestClass' in defs
assert defs['TestClass']['type'] == 'object'
assert 'properties' in defs['TestClass']

In [ ]:
# Test primitive types in containers
assert _handle_container(list, (int,), defs) == {'type': 'array', 'items': {'type': 'integer'}}
assert _handle_container(tuple, (str,), defs) == {'type': 'array', 'items': {'type': 'string'}}
assert _handle_container(set, (str,), defs) == {'type': 'array', 'items': {'type': 'string'}, 'uniqueItems': True}
assert _handle_container(dict, (str,bool), defs) == {'type': 'object', 'additionalProperties': {'type': 'boolean'}}

result = _handle_container(list, (TestClass,), defs)
assert result == {'type': 'array', 'items': {'$ref': '#/$defs/TestClass'}}
assert 'TestClass' in defs

# Test complex nested structure
ComplexType = dict[str, list[TestClass]]
result = _handle_container(dict, (str, list[TestClass]), defs)
assert result == {
    'type': 'object',
    'additionalProperties': {
        'type': 'array',
        'items': {'$ref': '#/$defs/TestClass'}
    }
}

In [ ]:
# Test processing of a required integer property
props, req = {}, {}
class TestClass:
    "Test class"
    def __init__(
        self,
        x: int, # First thing
        y: list[float], # Second thing
        z: str = "default", # Third thing
    ): store_attr()

d = docments(TestClass, full=True)
_process_property('x', d.x, props, req, defs)
assert 'x' in props
assert props['x']['type'] == 'integer'
assert 'x' in req

# Test processing of a required list property
_process_property('y', d.y, props, req, defs)
assert 'y' in props
assert props['y']['type'] == 'array'
assert props['y']['items']['type'] == 'number'
assert 'y' in req

# Test processing of an optional string property with default
_process_property('z', d.z, props, req, defs)
assert 'z' in props
assert props['z']['type'] == 'string'
assert props['z']['default'] == "default"
assert 'z' not in req

In [ ]:
#| export
def get_schema(
    f:Union[callable,dict], # Function to get schema for
    pname='input_schema',   # Key name for parameters
    evalable=False,  # stringify defaults that can't be literal_eval'd?
    skip_hidden=False # skip parameters starting with '_'?
)->dict: # {'name':..., 'description':..., pname:...}
    "Generate JSON schema for a class, function, or method"
    if isinstance(f, dict): return f
    schema = _get_nested_schema(f, evalable=evalable, skip_hidden=skip_hidden)
    desc = f.__doc__
    assert desc, "Docstring missing!"
    d = docments(f, full=True)
    ret = d.pop('return')
    has_type = (ret.anno is not empty) and (ret.anno is not None)
    has_doc = ret.docment
    if has_type or has_doc:
        type_str = f'type: {_types(ret.anno)[0]}' if has_type else None
        ret_str = f'{ret.docment} ({type_str})' if has_type and has_doc else (type_str if has_type else ret.docment)
        desc += f'\n\nReturns:\n- {ret_str}'
    return {"name": f.__name__, "description": desc, pname: schema}

In [ ]:
get_schema(get_schema)

{'name': 'get_schema',
 'description': "Generate JSON schema for a class, function, or method\n\nReturns:\n- {'name':..., 'description':..., pname:...} (type: object)",
 'input_schema': {'type': 'object',
  'properties': {'f': {'type': 'object',
    'description': 'Function to get schema for',
    'anyOf': [{'type': 'object'}, {'type': 'object'}]},
   'pname': {'type': 'string',
    'description': 'Key name for parameters',
    'default': 'input_schema'},
   'evalable': {'type': 'boolean',
    'description': "stringify defaults that can't be literal_eval'd?",
    'default': False},
   'skip_hidden': {'type': 'boolean',
    'description': "skip parameters starting with '_'?",
    'default': False}},
  'required': ['f']}}

### Usage examples

Putting this all together, we can now test getting a schema from `silly_sum`. The tool use spec doesn't support return annotations directly, so we put that in the description instead.

In [ ]:
s = get_schema(silly_sum)
desc = s.pop('description')
print(desc)
s

Adds a + b.

Returns:
- The sum of the inputs (type: integer)


{'name': 'silly_sum',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'First thing to sum'},
   'b': {'type': 'integer',
    'description': 'Second thing to sum',
    'default': 1},
   'c': {'type': 'array',
    'description': 'A pointless argument',
    'items': {'type': 'integer'},
    'default': None}},
  'required': ['a']}}

This also works with string annotations, e.g:

In [ ]:
def silly_test(
    a: 'int',  # quoted type hint
)->int:
    "Mandatory docstring"
    return a

get_schema(silly_test)

{'name': 'silly_test',
 'description': 'Mandatory docstring\n\nReturns:\n- type: integer',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'quoted type hint'}},
  'required': ['a']}}

This also works with instance methods:

In [ ]:
class Dummy:
    def sums(
        self,
        a:int,  # First thing to sum
        b:int=1 # Second thing to sum
    ): # The sum of the inputs
        "Adds a + b."
        print(f"Finding the sum of {a} and {b}")
        return a + b

get_schema(Dummy.sums)

{'name': 'sums',
 'description': 'Adds a + b.',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'First thing to sum'},
   'b': {'type': 'integer',
    'description': 'Second thing to sum',
    'default': 1}},
  'required': ['a']}}

`get_schema` also handles more complicated structures such as nested classes. This is useful for things like structured outputs.

In [ ]:
class Turn:
    "Turn between two speakers"
    def __init__(
        self,
        speaker_a:str, # First speaker's message
        speaker_b:str,  # Second speaker's message
    ): store_attr()

class Conversation:
    "A conversation between two speakers"
    def __init__(
        self,
        turns:list[Turn], # Turns of the conversation
    ): store_attr()

get_schema(Conversation)

{'name': 'Conversation',
 'description': 'A conversation between two speakers',
 'input_schema': {'type': 'object',
  'properties': {'turns': {'type': 'array',
    'description': 'Turns of the conversation',
    'items': {'$ref': '#/$defs/Turn'}}},
  'title': 'Conversation',
  'required': ['turns'],
  '$defs': {'Turn': {'type': 'object',
    'properties': {'speaker_a': {'type': 'string',
      'description': "First speaker's message"},
     'speaker_b': {'type': 'string',
      'description': "Second speaker's message"}},
    'title': 'Turn',
    'required': ['speaker_a', 'speaker_b']}}}}

In [ ]:
class DictConversation:
    "A conversation between two speakers"
    def __init__(
        self,
        turns:dict[str,list[Turn]], # dictionary of topics and the Turns of the conversation
    ): store_attr()

get_schema(DictConversation)

{'name': 'DictConversation',
 'description': 'A conversation between two speakers',
 'input_schema': {'type': 'object',
  'properties': {'turns': {'type': 'object',
    'description': 'dictionary of topics and the Turns of the conversation',
    'additionalProperties': {'type': 'array',
     'items': {'$ref': '#/$defs/Turn'}}}},
  'title': 'DictConversation',
  'required': ['turns'],
  '$defs': {'Turn': {'type': 'object',
    'properties': {'speaker_a': {'type': 'string',
      'description': "First speaker's message"},
     'speaker_b': {'type': 'string',
      'description': "Second speaker's message"}},
    'title': 'Turn',
    'required': ['speaker_a', 'speaker_b']}}}}

In [ ]:
class SetConversation:
    "A conversation between two speakers"
    def __init__(
        self,
        turns:set[Turn], # the unique Turns of the conversation
    ): store_attr()

get_schema(SetConversation)

{'name': 'SetConversation',
 'description': 'A conversation between two speakers',
 'input_schema': {'type': 'object',
  'properties': {'turns': {'type': 'array',
    'description': 'the unique Turns of the conversation',
    'items': {'$ref': '#/$defs/Turn'},
    'uniqueItems': True}},
  'title': 'SetConversation',
  'required': ['turns'],
  '$defs': {'Turn': {'type': 'object',
    'properties': {'speaker_a': {'type': 'string',
      'description': "First speaker's message"},
     'speaker_b': {'type': 'string',
      'description': "Second speaker's message"}},
    'title': 'Turn',
    'required': ['speaker_a', 'speaker_b']}}}}

### Additional `get_schema()` Test Cases

Union types are approximately mapped to JSON schema 'anyOf' with two or more value types.

In [ ]:
def _union_test(opt_tup: Union[Tuple[int, int], str, int]=None):
    "Mandatory docstring"
    return ""
get_schema(_union_test)

{'name': '_union_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'opt_tup': {'type': 'object',
    'description': '',
    'default': None,
    'anyOf': [{'type': 'array'}, {'type': 'string'}, {'type': 'integer'}]}}}}

The new (Python 3.10+) union syntax can also be used, producing an equivalent schema.

In [ ]:
def _new_union_test(opt_tup: Tuple[int, int] | str | int =None):
    "Mandatory docstring"
    pass
get_schema(_new_union_test)

{'name': '_new_union_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'opt_tup': {'type': 'object',
    'description': '',
    'default': None,
    'anyOf': [{'type': 'array'}, {'type': 'string'}, {'type': 'integer'}]}}}}

Optional is a special case of union types, limited to two types, one of which is None (mapped to null in JSON schema):

In [ ]:
def _optional_test(opt_tup: Optional[Tuple[int, int]]=None):
    "Mandatory docstring"
    pass
get_schema(_optional_test)

{'name': '_optional_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'opt_tup': {'type': 'object',
    'description': '',
    'default': None,
    'anyOf': [{'type': 'array'}, {'type': 'null'}]}}}}

Containers can also be used, both in their parameterized form (`List[int]`) or as their unparameterized raw type (`List`). In the latter case, the item type is mapped to `object` in JSON schema.

In [ ]:
def _list_test(l: List[int]):
    "Mandatory docstring"
    pass
get_schema(_list_test)

{'name': '_list_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'l': {'type': 'array',
    'description': '',
    'items': {'type': 'integer'}}},
  'required': ['l']}}

In [ ]:
def _raw_list_test(l: List):
    "Mandatory docstring"
    pass
get_schema(_raw_list_test)

{'name': '_raw_list_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'l': {'type': 'array',
    'description': '',
    'items': {'type': 'object'}}},
  'required': ['l']}}

The same applies to dictionary, which can similarly be parameterized with key/value types or specified as a raw type.

In [ ]:
def _dict_test(d: Dict[str, int]):
    "Mandatory docstring"
    pass
get_schema(_dict_test)

{'name': '_dict_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'d': {'type': 'object',
    'description': '',
    'additionalProperties': {'type': 'integer'}}},
  'required': ['d']}}

In [ ]:
def _raw_dict_test(d: Dict):
    "Mandatory docstring"
get_schema(_raw_dict_test)

{'name': '_raw_dict_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'d': {'type': 'object', 'description': ''}},
  'required': ['d']}}

In [ ]:
def _path_test(path: Path = Path('.')):
    "Mandatory docstring"
get_schema(_path_test)

{'name': '_path_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'path': {'type': 'string',
    'description': '',
    'default': Path('.'),
    'format': 'Path'}}}}

Schemas that need to be converted using `ast.literal_eval` will fail with non-primitive defaults:

In [ ]:
test_fail(lambda: ast.literal_eval(str(get_schema(_path_test))), exc=ValueError)

Use `evalable` to have those defaults stringified:

In [ ]:
def _path_test(path: Path = Path('.')):
    "Mandatory docstring"
get_schema(_path_test, evalable=True)

{'name': '_path_test',
 'description': 'Mandatory docstring',
 'input_schema': {'type': 'object',
  'properties': {'path': {'type': 'string',
    'description': '',
    'default': '.',
    'format': 'Path'}}}}

Use `skip_hidden` to exclude parameters starting with `_` from the schema:

In [ ]:
def test_hidden(a: int, _internal: str = "x"):
    "Test func"
    pass

get_schema(test_hidden, skip_hidden=True)  # should exclude _internal

{'name': 'test_hidden',
 'description': 'Test func',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': ''}},
  'required': ['a']}}

In [ ]:
get_schema(test_hidden)

{'name': 'test_hidden',
 'description': 'Test func',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': ''},
   '_internal': {'type': 'string', 'description': '', 'default': 'x'}},
  'required': ['a']}}

### Python tool

In language model clients it's often useful to have a 'code interpreter' -- this is something that runs code, and generally outputs the result of the last expression (i.e like IPython or Jupyter). 

In this section we'll create the `python` function, which executes a string as Python code, with an optional timeout. If the last line is an expression, we'll return that -- just like in IPython or Jupyter, but without needing them installed.

In [ ]:
#| exports
import ast, time, signal, traceback
from fastcore.utils import *

In [ ]:
#| exports
def _copy_loc(new, orig):
    "Copy location information from original node to new node and all children."
    new = ast.copy_location(new, orig)
    for field, o in ast.iter_fields(new):
        if isinstance(o, ast.AST): setattr(new, field, _copy_loc(o, orig))
        elif isinstance(o, list): setattr(new, field, [_copy_loc(value, orig) for value in o])
    return new

This is an internal function that's needed for `_run` to ensure that location information is available in the abstract syntax tree (AST), since otherwise python complains.

In [ ]:
#| exports
def _run(code:str, glb:dict=None, loc:dict=None):
    "Run `code`, returning final expression (similar to IPython)"
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgt = [ast.Name(id='_result', ctx=ast.Store())]
        assign_node = ast.Assign(targets=tgt, value=last_node.value)
        tree.body[-1] = _copy_loc(assign_node, last_node)

    compiled_code = compile(tree, filename='<ast>', mode='exec')
    glb = glb or {}
    stdout_buffer = io.StringIO()
    saved_stdout = sys.stdout
    sys.stdout = stdout_buffer
    try: exec(compiled_code, glb, loc)
    finally: sys.stdout = saved_stdout
    _result = glb.get('_result', None)
    if _result is not None: return _result
    return stdout_buffer.getvalue().strip()

This is the internal function used to actually run the code -- we pull off the last AST to see if it's an expression (i.e something that returns a value), and if so, we store it to a special `_result` variable so we can return it.

In [ ]:
_run('import math;math.factorial(12)')

479001600

In [ ]:
_run('print(1+1)')

'2'

We now have the machinery needed to create our `python` function.

In [ ]:
#| exports
def python(
    code:str, # Code to execute
    glb:Optional[dict]=None, # Globals namespace
    loc:Optional[dict]=None, # Locals namespace
    timeout:int=3600 # Maximum run time in seconds
):
    "Executes python `code` with `timeout` and returning final expression (similar to IPython)."
    def handler(*args): raise TimeoutError()
    if glb is None: glb = inspect.currentframe().f_back.f_globals
    if loc is None: loc=glb
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(timeout)
    try: return _run(code, glb, loc)
    except Exception as e: return traceback.format_exc()
    finally: signal.alarm(0)

There's no builtin security here -- you should generally use this in a sandbox, or alternatively prompt before running code. It can handle multiline function definitions, and pretty much any other normal Python syntax.

In [ ]:
python("""def factorial(n):
    if n == 0 or n == 1: return 1
    else: return n * factorial(n-1)
factorial(5)""")

120

If the code takes longer than `timeout` then it returns an error string.

In [ ]:
print(python('import time; time.sleep(10)', timeout=1))

Traceback (most recent call last):
  File "/var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_6265/2052945749.py", line 14, in python
    try: return _run(code, glb, loc)
                ^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_6265/1858893181.py", line 18, in _run
    try: exec(compiled_code, glb, loc)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ast>", line 1, in <module>
  File "/var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_6265/2052945749.py", line 9, in handler
    def handler(*args): raise TimeoutError()
                        ^^^^^^^^^^^^^^^^^^^^
TimeoutError



By default the caller's global namespace is used.

In [ ]:
python("a=1")
a

1

Pass a different `glb` if needed; this requires using `python_ns`.

In [ ]:
glb = {}
python("a=3", glb=glb)
a, glb['a']

(1, 3)

In [ ]:
get_schema(python)

{'name': 'python',
 'description': 'Executes python `code` with `timeout` and returning final expression (similar to IPython).',
 'input_schema': {'type': 'object',
  'properties': {'code': {'type': 'string', 'description': 'Code to execute'},
   'glb': {'type': 'object',
    'description': 'Globals namespace',
    'default': None,
    'anyOf': [{'type': 'object'}, {'type': 'null'}]},
   'loc': {'type': 'object',
    'description': 'Locals namespace',
    'default': None,
    'anyOf': [{'type': 'object'}, {'type': 'null'}]},
   'timeout': {'type': 'integer',
    'description': 'Maximum run time in seconds',
    'default': 3600}},
  'required': ['code']}}

### Tool Calling

Many LLM API providers offer tool calling where an LLM can choose to call a given tool. This is also helpful for structured outputs since the response from the LLM is contrained to the required arguments of the tool.

This section will be dedicated to helper functions for calling tools. We don't want to allow LLMs to call just any possible function (that would be a security disaster!) so we create a namespace -- that is, a dictionary of allowable function names to call.

In [ ]:
#| export
def mk_ns(fs):
    if isinstance(fs, abc.Mapping): return fs
    merged = {}
    for o in listify(fs):
        if isinstance(o, dict): merged |= o
        elif callable(o) and hasattr(o, '__name__'): merged |= {o.__name__: o}
    return merged

In [ ]:
def sums(a, b): return a + b
ns = mk_ns(sums); ns

{'sums': <function __main__.sums(a, b)>}

In [ ]:
ns['sums'](1, 2)

3

In [ ]:
#| export
def _coerce_inputs(func, inputs):
    "Coerce inputs based on function type annotations"
    hints = get_type_hints(func) if hasattr(func, '__annotations__') else {}
    res = {}
    for k,v in inputs.items():
        ann = hints.get(k)
        if ann in custom_types: res[k] = ann(v)
        elif isinstance(v, dict) and callable(ann): res[k] = ann(**v)
        else: res[k] = v
    return res

In [ ]:
#| export
def call_func(fc_name, fc_inputs, ns, raise_on_err=True):
    "Call the function `fc_name` with the given `fc_inputs` using namespace `ns`."
    if not isinstance(ns, abc.Mapping): ns = mk_ns(ns)
    func = ns[fc_name]
    inps = {re.sub(r'\W', '', k):v for k,v in fc_inputs.items()}
    inps = _coerce_inputs(func, inps)
    try: return func(**inps)
    except Exception as e:
        if raise_on_err: raise e from None
        else: return traceback.format_exc()

Now when we an LLM responses with the tool to use and its inputs, we can simply use the same namespace it was given to look up the tool and call it.

In [ ]:
call_func('sums', {'a': 1, 'b': 2}, ns=[sums])

3

In [ ]:
assert "unsupported operand type(s) for +: 'int' and 'str'" in call_func('sums', {'a': 1, 'b': '3'}, ns=ns, raise_on_err=False)

In [ ]:
test_fail(call_func, args=['sums', {'a': 1, 'b': '3'}], kwargs={'ns': ns})

Types that can be constructed from a plain `str` can be used directly, as long as they are in `custom_types` (which you can add to).

In [ ]:
def path_test(
    a: Path,  # a type hint
    b: Path   # b type hint
):
    "Mandatory docstring"
    return a/b

test_eq(call_func('path_test', {'a': '/home', 'b': 'user'}, ns=[path_test]), Path('/home/user'))

### Async function calling

In [ ]:
async def asums(a, b): return a + b
ns = mk_ns(asums); ns

{'asums': <function __main__.asums(a, b)>}

In [ ]:
#| exports
async def call_func_async(fc_name, fc_inputs, ns, raise_on_err=True):
    "Awaits the function `fc_name` with the given `fc_inputs` using namespace `ns`."
    res = call_func(fc_name, fc_inputs, ns, raise_on_err=raise_on_err)
    if inspect.iscoroutine(res):
        try: res = await res
        except Exception as e:
            if raise_on_err: raise e from None
            else: return traceback.format_exc()
    return res

In [ ]:
await call_func_async('asums', {'a': 1, 'b': 2}, ns=[asums])

3

In [ ]:
r = await call_func_async('asums', {'a': 1, 'b': '2'}, ns=[asums], raise_on_err=False)
assert "unsupported operand type(s) for +: 'int' and 'str'" in r

In [ ]:
ex = False
try: await call_func_async('asums', {'a': 1, 'b': '2'}, ns=[asums], raise_on_err=True)
except: ex = True
assert ex

## Schema to function

In [ ]:
type_map = {'string': str, 'boolean': bool, 'integer': int, 'number': float, 'array': list, 'object': dict}

In [ ]:
#| export
def mk_param(nm, props, req):
    "Create a `Parameter` for `nm` with schema `props`"
    kind = Parameter.POSITIONAL_OR_KEYWORD if nm in req else Parameter.KEYWORD_ONLY
    default = Parameter.empty if nm in req else props.get('default')    
    if props.get('type') == 'array' and 'items' in props:
        item_type = type_map.get(props['items'].get('type'), Any)
        anno = list[item_type]
    else: anno = type_map.get(props.get('type'), Any)
    return Parameter(nm, kind, default=default, annotation=anno)

In [ ]:
tool = dict2obj({
    'description': 'Find real-…',
    'inputSchema': { '$schema': 'http://json-schema.org/draft-07/schema#',
                   'additionalProperties': False,
                   'properties': { 'language': { 'description': 'Filter by …', 'items': {'type': 'string'}, 'type': 'array'},
                                   'matchCase': {'default': False, 'description': 'Whether th…', 'type': 'boolean'},
                                   'path': {'description': 'Filter by …', 'type': 'string'},
                                   'query': {'description': 'The litera…', 'type': 'string'},
                                   'useRegexp': {'default': False, 'description': 'Whether to…', 'type': 'boolean'}},
                   'required': ['query'], 'type': 'object'},
    'name': 'searchGitHub'
})

In [ ]:
props, req = tool.inputSchema['properties'], tool.inputSchema['required']
list(props)

['language', 'matchCase', 'path', 'query', 'useRegexp']

In [ ]:
props.matchCase

```python
{'default': False, 'description': 'Whether th…', 'type': 'boolean'}
```

In [ ]:
p = mk_param('query', props.query, req)
p, p.kind

(<Parameter "query: str">, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)

In [ ]:
p = mk_param('language', props.language, req)
p, p.kind

(<Parameter "language: list[str] = None">, <_ParameterKind.KEYWORD_ONLY: 3>)

In [ ]:
#| export
def schema2sig(tool):
    "Convert json schema `tool` to a `Signature`"
    props, req = tool.inputSchema['properties'], tool.inputSchema.get('required', [])
    params = sorted([mk_param(k, v, req) for k, v in props.items()], key=lambda p: p.kind)
    return Signature(params)

In [ ]:
schema2sig(tool)

<Signature (query: str, *, language: list[str] = None, matchCase: bool = False, path: str = None, useRegexp: bool = False)>

In [ ]:
#| export
def mk_tool(dispfn, tool):
    "Create a callable function from a JSON schema tool definition"
    sig = schema2sig(tool)
    props = tool.inputSchema['properties']
    def fn(*args, **kwargs):
        bound = sig.bind(*args, **kwargs)
        return dispfn(tool.name, **bound.arguments)
    fn.__doc__ = tool.description
    fn.__signature__ = sig
    fn.__name__ = fn.__qualname__ = tool.name
    fn.__annotations__ = {k: p.annotation for k, p in sig.parameters.items()}
    return fn

`mk_tool` is the inverse of `get_schema` — it creates a callable Python function from a JSON schema tool definition. This is useful for MCP clients where tools are defined as schemas but need to be called as regular Python functions.

The created function has a proper signature, docstring, and annotations, so it works well with IDE autocomplete and introspection.

In [ ]:
def dispatch_eg(name, **kwargs): return f"Called {name} with {kwargs}"

fn = mk_tool(dispatch_eg, tool)
fn('hello', path='src/')

"Called searchGitHub with {'query': 'hello', 'path': 'src/'}"

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()